In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import re
import time
import openai
import pickle
import numpy as np
import pandas as pd
import string
import random

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
from scipy import stats
from scipy.stats import sem 
import itertools

np.seterr(divide = 'ignore') 
np.seterr(divide = 'warn') 

%load_ext autoreload
%autoreload 2

In [2]:
target_dir = 'results'

In [3]:
target_var_list = ['GRDP', 'population', 'education_high']
target_ccode = 'KOR' # KOR / MWI / KHM

In [4]:
num_shot = 3 # 3 / 5 shot
num_valid = 1

In [15]:
mapping = {
    'unemployment_rate': 'UNEMP_T'
}

country_name_mapping = {
    'KOR': 'South Korea',
    'VNM': 'Viet Nam',
    'KHM': 'Cambodia'
}

file_path_mapping = {
    'GRDP': 'GRDP',
    'population': 'Population',
    'education_high': 'Education'
}

target_var_desc_mapping = {
    'GRDP': 'a regional GDP',
    'population': 'the population',
    'education_high': 'the proportion of highly-educated individuals (at least university graduates)'
}

target_var_detailed_desc_mapping = {
    'GRDP': 'the total economic output of a specific region within a country, reflecting the value of all goods and services over a specific period.',
    'population': 'a group of individuals or entities of the same species living in a specific area or considered for a study.',
    'education_high': 'the percentage of a population that has attained at least a university-level degree.'
}

module_mapping = {
    'get_address(Loc)': ['address'],
    'get_area(Loc)': ['area'],
    'get_distance_to_nearest_target(Loc, "airport")': ['distance_airport'],
    'get_distance_to_nearest_target(Loc, "port")': ['distance_port'],
    'get_night_light(Loc)': ['Nightlight_Sum', 'Nightlight_Average'],
    'count_area(Loc, "agricultural")': ['area_agricultural'],
    'count_area(Loc, "bareland")': ['area_bareland'],
    'count_area(Loc, "building")': ['area_building'],
    'count_area(Loc, "development")': ['area_development'],
    'count_area(Loc, "rangeland")': ['area_rangeland'],
    'count_area(Loc, "road")': ['area_road'],
    'count_area(Loc, "tree")': ['area_tree'],
    'count_area(Loc, "water")': ['area_water'],
    'get_aggregate_neighbor_info(Loc, lambda x: get_night_light(loc=x))': ['neighbor_Nightlight_Sum', 
                                                                           'neighbor_Nightlight_Average'],
    'get_aggregate_neighbor_info(Loc, lambda x: get_area(loc=x))': ['neighbor_area'],
    'get_aggregate_neighbor_info(Loc, lambda x: get_distance_to_nearest_target(loc=x, "airport"))': ['neighbor_distance_airport'],
    'get_aggregate_neighbor_info(Loc, lambda x: get_distance_to_nearest_target(loc=x, "port"))': ['neighbor_distance_port'],
    'get_aggregate_neighbor_info(Loc, lambda x: count_area(loc=x, "agricultural"))': ['neighbor_area_agricultural'], 
    'get_aggregate_neighbor_info(Loc, lambda x: count_area(loc=x, "bareland"))': ['neighbor_area_bareland'], 
    'get_aggregate_neighbor_info(Loc, lambda x: count_area(loc=x, "building"))': ['neighbor_area_building'], 
    'get_aggregate_neighbor_info(Loc, lambda x: count_area(loc=x, "development"))': ['neighbor_area_development'], 
    'get_aggregate_neighbor_info(Loc, lambda x: count_area(loc=x, "rangeland"))': ['neighbor_area_rangeland'], 
    'get_aggregate_neighbor_info(Loc, lambda x: count_area(loc=x, "road"))': ['neighbor_area_road'], 
    'get_aggregate_neighbor_info(Loc, lambda x: count_area(loc=x, "tree"))': ['neighbor_area_tree'], 
    'get_aggregate_neighbor_info(Loc, lambda x: count_area(loc=x, "water"))': ['neighbor_area_water'],
}

module_desc_mapping = {
    'address': 'address',
    'area': 'area size',
    'distance_airport': 'distance to airport',
    'distance_port': 'distance to port',
    'Nightlight_Sum': 'sum of nightlight intensity',
    'Nightlight_Average': 'average of nightlight intensity',
    'area_agricultural': 'ratio of agricultural area',
    'area_bareland': 'ratio of bareland area',
    'area_building': 'ratio of building area',
    'area_development': 'ratio of development area (e.g., sidewalks, pavements, footpaths, and artificial grass areas from tennis courts and baseball fields)',
    'area_rangeland': 'ratio of rangeland area',
    'area_road': 'ratio of road area',
    'area_tree': 'ratio of tree area',
    'area_water': 'ratio of water area',
    'neighbor_Nightlight_Sum': 'sum of nightlight intensity of surrounding regions',
    'neighbor_Nightlight_Average': 'average of nightlight intensity of surrounding regions',
    'neighbor_area': 'average of area size of surrounding regions',
    'neighbor_distance_airport': 'average of disatance of airport of surrounding regions',
    'neighbor_distance_port': 'average of disatance of port of surrounding regions',
    'neighbor_area_agricultural': 'ratio of agricultural area of surrounding regions',
    'neighbor_area_bareland': 'ratio of bareland area of surrounding regions',
    'neighbor_area_building': 'ratio of building area of surrounding regions',
    'neighbor_area_development': 'ratio of development area of surrounding regions (e.g., sidewalks, pavements, footpaths, and artificial grass areas from tennis courts and baseball fields)',
    'neighbor_area_rangeland': 'ratio of rangeland area of surrounding regions',
    'neighbor_area_road': 'ratio of road area of surrounding regions',
    'neighbor_area_tree': 'ratio of tree area of surrounding regions',
    'neighbor_area_water': 'ratio of water area of surrounding regions',
}

module_detailed_desc_mapping = {
    'area': 'the area size of a given region',
    'distance_airport': 'the distance to airport of a given region',
    'distance_port': 'the distance to port of given location of the region',
    'nightlight': 'the sum/average of nightlight intensity of a given region',
    'area_agricultural': 'the ratio of agricultural area of a given region',
    'area_bareland': 'the ratio of bareland area of a given region',
    'area_building': 'the ratio of building area of a given region',
    'area_development': 'the ratio of development area of a given region (i.e., development involves areas such as sidewalks, pavements, footpaths, and artificial grass areas from tennis courts and baseball fields)',
    'area_rangeland': 'the ratio of rangeland area of a given region',
    'area_road': 'the ratio of road area of a given region',
    'area_tree': 'the ratio of tree area of a given region',
    'area_water': 'the ratio of water area of a given region',
    'neighbor_nightlight': 'the sum/average of nightlight intensity of surrounding regions',
    'neighbor_area': 'the average of area size of surrounding regions',
    'neighbor_distance_airport': 'the average of disatance of airport of surrounding regions',
    'neighbor_distance_port': 'the average of disatance of port of surrounding regions',
    'neighbor_area_agricultural': 'the ratio of agricultural area of surrounding regions',
    'neighbor_area_bareland': 'the ratio of bareland area of surrounding regions',
    'neighbor_area_building': 'the ratio of building area of surrounding regions',
    'neighbor_area_development': 'the ratio of development area of surrounding regions (i.e., development involves areas such as sidewalks, pavements, footpaths, and artificial grass areas from tennis courts and baseball fields)',
    'neighbor_area_rangeland': 'the ratio of rangeland area of surrounding regions',
    'neighbor_area_road': 'the ratio of road area of surrounding regions',
    'neighbor_area_tree': 'the ratio of tree area of surrounding regions',
    'neighbor_area_water': 'the ratio of water area of surrounding regions',
}

module_desc_for_incontext_mapping = {
    'address': 'Full address of a given region',
    'area': 'Area of district (km^2) of a given region',
    'distance_airport': 'Distance to the nearest airport of a given region',
    'distance_port': 'Distance to the nearest port of a given region',
    'Nightlight_Sum': 'Sum nightlight intensity of a given region',
    'Nightlight_Average': 'Average nightlight intensity of a given region',
    'area_agricultural': 'Land cover ratio of agricultural of a given region',
    'area_bareland': 'Land cover ratio of bareland of a given region',
    'area_building': 'Land cover ratio of building of a given region',
    'area_development': 'Land cover ratio of development of a given region',
    'area_rangeland': 'Land cover ratio of rangeland of a given region',
    'area_road': 'Land cover ratio of road of a given region',
    'area_tree': 'Land cover ratio of tree of a given region',
    'area_water': 'Land cover ratio of water of a given region',
    'neighbor_Nightlight_Sum': 'Sum nightlight intensity in surrounding regions',
    'neighbor_Nightlight_Average': 'Average nightlight intensity in surrounding regions',
    'neighbor_area': 'Average of area of district (km^2) in surrounding regions',
    'neighbor_distance_airport': 'Average of disatance to the nearest airport in surrounding regions',
    'neighbor_distance_port': 'Average of disatance to the nearest port in surrounding regions',
    'neighbor_area_agricultural': 'Land cover ratio of agricultural in surrounding regions',
    'neighbor_area_bareland': 'Land cover ratio of bareland in surrounding regions',
    'neighbor_area_building': 'Land cover ratio of building in surrounding regions',
    'neighbor_area_development': 'Land cover ratio of development in surrounding regions',
    'neighbor_area_rangeland': 'Land cover ratio of rangeland in surrounding regions',
    'neighbor_area_road': 'Land cover ratio of road in surrounding regions',
    'neighbor_area_tree': 'Land cover ratio of tree in surrounding regions',
    'neighbor_area_water': 'Land cover ratio of area in surrounding regions',
}

In [17]:
def ridge_objective_function(coeffs, X, y, lmbda):
    predictions = X @ coeffs[:-1] + coeffs[-1]
    mse = ((y - predictions) ** 2).mean()
    ridge_penalty = lmbda * (coeffs[:-1] ** 2).sum()
    return mse + ridge_penalty

In [18]:
def interaction(answers, temp_df):
    
    feature_list = answers
    
    new_dict = {}
    column_names = []
    
    column_id = 0
    
    for idx, (feature_type, feature) in enumerate (feature_list):
        ori_feature = feature
        try:
            new_column_name = f'new {column_id}'
            alpha = string.ascii_lowercase
            index = 0 
            memory = {}

            if 'Neighbor_Nightlight_Sum' in feature:
                feature = feature.replace('Neighbor_Nightlight_Sum', 'neighbor_Nightlight_Sum')
            if 'Neighbor_Nightlight_Average' in feature:
                feature = feature.replace('Neighbor_Nightlight_Average', 'neighbor_Nightlight_Average')
                
            # exception
            if 'np.log(distance_airport * neighbor_area' in feature:
                feature = feature.replace('np.log(distance_airport * neighbor_area', 'np.log(distance_airport * neighbor_area)')
            
            for column in list(all_df.columns)[::-1]:
                feature = feature.replace(f'np.{column}', column)
                
            for column in list(all_df.columns)[::-1]:
                if column in feature:
                    feature = feature.replace(f'{column}', f'temp_df[memory[\'{alpha[index]}\']]')
                    memory[alpha[index]] = column
                    index += 1

            new_dict[new_column_name] = eval(feature)
            column_names.append(ori_feature)
            
            column_id += 1
            
        except:
            assert(0)
        
    new_df = pd.DataFrame(new_dict)
    new_df.columns = column_names
        
    return new_df

In [9]:
target_success_seed_list = []
target_success_ensemble_list = []

In [21]:
for ensemble_set in [25]:
    
    spearman_mean_list = []
    spearman_sem_list = []
    pearson_mean_list = []
    pearson_sem_list = []
    r2_mean_list = []
    r2_sem_list = []
    rmse_mean_list = []
    rmse_sem_list = []
        
    for target_index, target_var in enumerate (target_var_list):
    
        target_var = target_var_list[target_index]
        
        success_seed_list = []
        success_ensemble_list = []

        max_try_num = 1
        # ensemble_num = [10, 15, 20, 25, 30, 35, 40]
        # max_ensemble_set = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
        max_ensemble_set = [ensemble_set]
        # max_interaction_terms_set = [10, 15, 20, 25, 30, 35, 40]
        
        answers_path = f'./data/{target_dir}/{target_ccode}_{target_var}_extension_paragraph_output_25.csv'
        answers_df = pd.read_csv(answers_path)
        
        if target_ccode == 'VNM':
            adm_offset = '_adm1'
        else:
            adm_offset = ''

        if (target_ccode != 'KOR' and target_ccode != 'VNM') and target_var == 'GRDP':
            assert(0)
        elif target_ccode == 'KOR' and target_var == 'literacy_rate':
            assert(0)

        if target_var in list(mapping.keys()):
            col_var = mapping[target_var]
        else:
            col_var = target_var

        file_path_var = file_path_mapping[target_var]

        if target_var == 'literacy_rate' and target_ccode == 'VNM':
            col_var = 'LIT_15'
        elif target_var == 'literacy_rate' and target_ccode == 'KHM':
            col_var = 'LIT_T'

        target_var_desc = target_var_desc_mapping[target_var]
        country_name = country_name_mapping[target_ccode]
        
        Inference_desc = f'Estimate {target_var_desc} of a given district. Answer the numeric score only'
        TASK_DESC = f'Estimate {target_var_desc} of a given district in {country_name}.'
        
        file_path_var = file_path_mapping[target_var]

        gt = pd.read_csv(f'./data/labels/{target_ccode}_{file_path_var}{adm_offset}.csv', encoding='utf-8')[['area_id', col_var]]
        all_results_path = f'./data/all_features/{target_ccode}.csv'
        all_df = pd.read_csv(all_results_path)
        
        if target_var == 'GRDP' or target_var == 'population' or target_var == 'age_working_population':
            gt[col_var] = np.log(gt[col_var])
        elif target_var == 'age_old_dependency_ratio' or target_var == 'infant_mortality':
            gt[col_var] = (gt[col_var])/1000.0
        else:
            pass
        
        gpt35_results = []
        base_results = []
        no_results = []
        pos_results = []
        pos_neg_results = []
        pos_neg_mix_results = []
        
        for max_ensemble in max_ensemble_set:
    
            reverse_ensemble_id = 25
            all_results = []

            for random_seed in [111, 222, 333, 444, 555]:

                random.seed(random_seed)

                test_val_list = []
                for ensemble_id, ensemble in enumerate (range(max_ensemble)):
                    print(f"Seed: {random_seed}, ensemble num: {ensemble}")
                    try_num = 0
#                     current_seed = random_seed
                    while try_num < max_try_num:
                        try:
                
                            module_dict = eval(answers_df[answers_df['seed'] == random_seed].iloc[ensemble_id]['module_dict'])

                            assert(len(module_dict) > 0)

                            module_columns = {}
                            for key, val in module_dict.items():
                                flag = False
                                s_columns = []
                                new_s_m = []
                                for s_m in val:
                                    if s_m == 'nightlight':
                                        s_columns.append('Nightlight_Sum')
                                        s_columns.append('Nightlight_Average')
                                        new_s_m.append('Nightlight_Sum')
                                        new_s_m.append('Nightlight_Average')
                                    elif s_m == 'neighbor_nightlight':
                                        s_columns.append('neighbor_Nightlight_Sum')
                                        s_columns.append('neighbor_Nightlight_Average')
                                        new_s_m.append('neighbor_Nightlight_Sum')
                                        new_s_m.append('neighbor_Nightlight_Average')
                                    else:
                                        s_columns.append(s_m)
                                        new_s_m.append(s_m)
                                    flag = True

                                module_dict[key] = new_s_m

                                if flag == False:
                                    print("error val")
                                    print(val)
                                    assert(0)
                                module_columns[key] = s_columns

                            address_df = all_df['address']
                            positive_df = all_df[module_columns['positive']]
                            negative_df = all_df[module_columns['negative']]
                            mixed_df = all_df[module_columns['mixed']]

                            positive_answers = eval(answers_df[answers_df['seed'] == random_seed].iloc[ensemble_id]['pos_list'])
                            negative_answers = eval(answers_df[answers_df['seed'] == random_seed].iloc[ensemble_id]['neg_list'])
                            mixed_answers = eval(answers_df[answers_df['seed'] == random_seed].iloc[ensemble_id]['mixed_list'])

                            if positive_answers == -1:
                                assert(0)
                            if negative_answers == -1:
                                assert(0)
                            if mixed_answers == -1:
                                assert(0)
                                
                            print(random_seed, ensemble_id)
                            print(positive_answers)

                            inter_positive_df = interaction(positive_answers, positive_df)
                            inter_negative_df = interaction(negative_answers, negative_df)
                            inter_mixed_df = interaction(mixed_answers, mixed_df)

                            inter_positive_df = inter_positive_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                            inter_negative_df = inter_negative_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                            inter_mixed_df = inter_mixed_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

                            df_all = pd.concat([positive_df, inter_positive_df, -negative_df, -inter_negative_df, mixed_df, inter_mixed_df], axis=1)
                            scalar = StandardScaler() 
                            df_all = scalar.fit_transform(df_all)

                            df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(df_all, gt[col_var].to_numpy(), train_size=num_shot, random_state=random_seed)
                            initial_coeffs = np.ones(df_all.shape[1]+1)
                            lambda_ridge = 1.0
                            bounds = []
                            for i in range(df_all.shape[1]+1):
                                if i < positive_df.shape[1] + inter_positive_df.shape[1] + negative_df.shape[1] + inter_negative_df.shape[1]:
                                    bounds.append((0, None))
                                else:
                                    bounds.append((None, None))

                            result = minimize(ridge_objective_function, initial_coeffs, args=(df_x_train, df_y_train, lambda_ridge), 
                                              method='SLSQP', bounds=bounds, options={'maxiter': 2000})

                            if result.success == False:
                                print("Optimization failed.")
                                print("Reason:", result.message)

                            success_seed_list.append(random_seed)
                            success_ensemble_list.append(ensemble_id)
                            
                            print("test again")
                            print(random_seed, ensemble_id)
                            
                            break

                        except:
                            print(f"failed - {try_num}/{max_try_num}")
                            try_num += 1
                            continue
        target_success_seed_list.append(success_seed_list)
        target_success_ensemble_list.append(success_ensemble_list)

Seed: 111, ensemble num: 0
error val
[]
failed - 0/1
Seed: 111, ensemble num: 1
error val
[]
failed - 0/1
Seed: 111, ensemble num: 2
error val
[]
failed - 0/1
Seed: 111, ensemble num: 3
111 3
[('Interaction between area and Nightlight_Sum', '(area)*(Nightlight_Sum)'), ('Interaction between Nightlight_Average and area_building', '(Nightlight_Average)*(area_building)'), ('Interaction between area_development and area_road', '(area_development)*(area_road)'), ('Interaction between area_tree and area_water', '(area_tree)*(area_water)'), ('Interaction between neighbor_Nightlight_Sum and neighbor_Nightlight_Average', '(neighbor_Nightlight_Sum)*(neighbor_Nightlight_Average)'), ('Interaction between neighbor_area and neighbor_area_bareland', '(neighbor_area)*(neighbor_area_bareland)'), ('Interaction between neighbor_area_building and neighbor_area_development', '(neighbor_area_building)*(neighbor_area_development)'), ('Interaction between neighbor_area_rangeland and neighbor_area_road', '(neig

test again
111 13
Seed: 111, ensemble num: 14
111 14
[('Interaction between area and Nightlight_Sum', '(area)*(Nightlight_Sum)'), ('Interaction between Nightlight_Average and area_building', '(Nightlight_Average)*(area_building)'), ('Interaction between area_development and area_road', '(area_development)*(area_road)'), ('Interaction between area_tree and area_water', '(area_tree)*(area_water)'), ('Interaction between neighbor_Nightlight_Sum and neighbor_Nightlight_Average', '(neighbor_Nightlight_Sum)*(neighbor_Nightlight_Average)'), ('Interaction between neighbor_area and neighbor_area_bareland', '(neighbor_area)*(neighbor_area_bareland)'), ('Interaction between neighbor_area_building and neighbor_area_development', '(neighbor_area_building)*(neighbor_area_development)'), ('Interaction between neighbor_area_rangeland and neighbor_area_road', '(neighbor_area_rangeland)*(neighbor_area_road)'), ('Interaction between neighbor_area_tree and neighbor_area_water', '(neighbor_area_tree)*(neig

test again
222 7
Seed: 222, ensemble num: 8
error val
[]
failed - 0/1
Seed: 222, ensemble num: 9
error val
[]
failed - 0/1
Seed: 222, ensemble num: 10
222 10
[('Interaction between area and Nightlight_Sum', '(area)*(Nightlight_Sum)'), ('Interaction between area and Nightlight_Average', '(area)*(Nightlight_Average)'), ('Interaction between area_building and area_development', '(area_building)*(area_development)'), ('Interaction between area_building and area_road', '(area_building)*(area_road)'), ('Interaction between area_building and area_tree', '(area_building)*(area_tree)'), ('Interaction between area_building and area_water', '(area_building)*(area_water)'), ('Interaction between area_building and neighbor_Nightlight_Sum', '(area_building)*(neighbor_Nightlight_Sum)'), ('Interaction between area_building and neighbor_Nightlight_Average', '(area_building)*(neighbor_Nightlight_Average)'), ('Interaction between area_building and neighbor_area', '(area_building)*(neighbor_area)'), ('Int

test again
222 14
Seed: 222, ensemble num: 15
222 15
[('Interaction between area and Nightlight_Sum', '(area)*(Nightlight_Sum)'), ('Interaction between Nightlight_Average and area_building', '(Nightlight_Average)*(area_building)'), ('Interaction between area_development and area_road', '(area_development)*(area_road)'), ('Interaction between area_tree and area_water', '(area_tree)*(area_water)'), ('Interaction between neighbor_Nightlight_Sum and neighbor_Nightlight_Average', '(neighbor_Nightlight_Sum)*(neighbor_Nightlight_Average)'), ('Interaction between neighbor_area and neighbor_area_bareland', '(neighbor_area)*(neighbor_area_bareland)'), ('Interaction between neighbor_area_building and neighbor_area_development', '(neighbor_area_building)*(neighbor_area_development)'), ('Interaction between neighbor_area_rangeland and neighbor_area_road', '(neighbor_area_rangeland)*(neighbor_area_road)'), ('Interaction between neighbor_area_tree and neighbor_area_water', '(neighbor_area_tree)*(neig

test again
333 0
Seed: 333, ensemble num: 1
333 1
[('Interaction between area and Nightlight_Sum', '(area)*(Nightlight_Sum)'), ('Interaction between Nightlight_Average and area_building', '(Nightlight_Average)*(area_building)'), ('Interaction between area_development and area_road', '(area_development)*(area_road)'), ('Interaction between area_tree and area_water', '(area_tree)*(area_water)'), ('Interaction between neighbor_Nightlight_Sum and neighbor_Nightlight_Average', '(neighbor_Nightlight_Sum)*(neighbor_Nightlight_Average)'), ('Interaction between neighbor_area and neighbor_area_bareland', '(neighbor_area)*(neighbor_area_bareland)'), ('Interaction between neighbor_area_building and neighbor_area_development', '(neighbor_area_building)*(neighbor_area_development)'), ('Interaction between neighbor_area_rangeland and neighbor_area_road', '(neighbor_area_rangeland)*(neighbor_area_road)'), ('Interaction between neighbor_area_tree and neighbor_area_water', '(neighbor_area_tree)*(neighbo

...

test again
444 13
Seed: 444, ensemble num: 14
444 14
[('Interaction between area and Nightlight_Sum', '(area)*(Nightlight_Sum)'), ('Interaction between Nightlight_Average and area_building', '(Nightlight_Average)*(area_building)'), ('Interaction between area_development and area_road', '(area_development)*(area_road)'), ('Interaction between area_tree and area_water', '(area_tree)*(area_water)'), ('Interaction between neighbor_Nightlight_Sum and neighbor_Nightlight_Average', '(neighbor_Nightlight_Sum)*(neighbor_Nightlight_Average)'), ('Interaction between neighbor_area and neighbor_area_bareland', '(neighbor_area)*(neighbor_area_bareland)'), ('Interaction between neighbor_area_building and neighbor_area_development', '(neighbor_area_building)*(neighbor_area_development)'), ('Interaction between neighbor_area_rangeland and neighbor_area_road', '(neighbor_area_rangeland)*(neighbor_area_road)'), ('Interaction between neighbor_area_tree and neighbor_area_water', '(neighbor_area_tree)*(neig

test again
555 2
Seed: 555, ensemble num: 3
555 3
[('Interaction between area and Nightlight_Sum', '(area)*(Nightlight_Sum)'), ('Interaction between area and Nightlight_Average', '(area)*(Nightlight_Average)'), ('Interaction between Nightlight_Sum and Nightlight_Average', '(Nightlight_Sum)*(Nightlight_Average)'), ('Interaction between area_building and area_development', '(area_building)*(area_development)'), ('Interaction between area_building and area_road', '(area_building)*(area_road)'), ('Interaction between area_building and area_tree', '(area_building)*(area_tree)'), ('Interaction between area_building and area_water', '(area_building)*(area_water)'), ('Interaction between area_development and area_road', '(area_development)*(area_road)'), ('Interaction between area_development and area_tree', '(area_development)*(area_tree)'), ('Interaction between area_development and area_water', '(area_development)*(area_water)'), ('Interaction between area_road and area_tree', '(area_road)*

test again
555 10
Seed: 555, ensemble num: 11
555 11
[('Interaction between area and Nightlight_Sum', '(area)*(Nightlight_Sum)'), ('Interaction between Nightlight_Average and area_building', '(Nightlight_Average)*(area_building)'), ('Interaction between area_development and area_road', '(area_development)*(area_road)'), ('Interaction between area_tree and area_water', '(area_tree)*(area_water)'), ('Interaction between neighbor_Nightlight_Sum and neighbor_Nightlight_Average', '(neighbor_Nightlight_Sum)*(neighbor_Nightlight_Average)'), ('Interaction between neighbor_area and neighbor_area_bareland', '(neighbor_area)*(neighbor_area_bareland)'), ('Interaction between neighbor_area_building and neighbor_area_development', '(neighbor_area_building)*(neighbor_area_development)'), ('Interaction between neighbor_area_rangeland and neighbor_area_road', '(neighbor_area_rangeland)*(neighbor_area_road)'), ('Interaction between neighbor_area_tree and neighbor_area_water', '(neighbor_area_tree)*(neig

test again
555 13
Seed: 555, ensemble num: 14
555 14
[('Interaction between area and Nightlight_Sum', '(area)*(Nightlight_Sum)'), ('Interaction between area and Nightlight_Average', '(area)*(Nightlight_Average)'), ('Interaction between area_building and area_development', '(area_building)*(area_development)'), ('Interaction between area_building and area_road', '(area_building)*(area_road)'), ('Interaction between area_building and area_tree', '(area_building)*(area_tree)'), ('Interaction between area_building and area_water', '(area_building)*(area_water)'), ('Interaction between area_development and area_road', '(area_development)*(area_road)'), ('Interaction between area_development and area_tree', '(area_development)*(area_tree)'), ('Interaction between area_development and area_water', '(area_development)*(area_water)'), ('Interaction between area_road and area_tree', '(area_road)*(area_tree)'), ('Interaction between area_road and area_water', '(area_road)*(area_water)'), ('Intera

test again
555 19
Seed: 555, ensemble num: 20
failed - 0/1
Seed: 555, ensemble num: 21
failed - 0/1
Seed: 555, ensemble num: 22
failed - 0/1
Seed: 555, ensemble num: 23
failed - 0/1
Seed: 555, ensemble num: 24
failed - 0/1


In [22]:
print(target_success_seed_list)
print(target_success_ensemble_list)
print(len(target_success_ensemble_list))
print(len(target_success_seed_list[0]))

[[111, 111, 111, 111, 111, 111, 111, 222, 222, 222, 222, 222, 222, 222, 222, 222, 333, 333, 333, 333, 333, 333, 333, 333, 333, 333, 333, 333, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 555, 555, 555, 555, 555, 555, 555, 555, 555, 555, 555, 555, 555]]
[[3, 9, 11, 13, 14, 15, 17, 3, 7, 10, 12, 13, 14, 16, 17, 19, 0, 2, 3, 4, 5, 6, 8, 9, 12, 13, 18, 19, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 0, 2, 3, 7, 8, 10, 11, 12, 13, 14, 15, 16, 19]]
1
54


# Get the three(111, 222, 333) samples with lowest sem

In [23]:
for ensemble_set in [1]:
    
    spearman_mean_list = []
    spearman_sem_list = []
    pearson_mean_list = []
    pearson_sem_list = []
    r2_mean_list = []
    r2_sem_list = []
    rmse_mean_list = []
    rmse_sem_list = []
    
    total_best_dict = {}
        
    for target_index, target_var in enumerate (target_var_list):
    
        target_var = target_var_list[target_index]

        max_try_num = 1
        max_ensemble_set = [ensemble_set]
        
        answers_path = f'./data/{target_dir}/{target_ccode}_{target_var}_extension_paragraph_output_25.csv'
        answers_df = pd.read_csv(answers_path)
        
        if target_ccode == 'VNM':
            adm_offset = '_adm1'
        else:
            adm_offset = ''

        if (target_ccode != 'KOR' and target_ccode != 'VNM') and target_var == 'GRDP':
            assert(0)
        elif target_ccode == 'KOR' and target_var == 'literacy_rate':
            assert(0)

        if target_var in list(mapping.keys()):
            col_var = mapping[target_var]
        else:
            col_var = target_var

        file_path_var = file_path_mapping[target_var]

        if target_var == 'literacy_rate' and target_ccode == 'VNM':
            col_var = 'LIT_15'
        elif target_var == 'literacy_rate' and target_ccode == 'KHM':
            col_var = 'LIT_T'

        target_var_desc = target_var_desc_mapping[target_var]
        country_name = country_name_mapping[target_ccode]
        
        Inference_desc = f'Estimate {target_var_desc} of a given district. Answer the numeric score only'
        TASK_DESC = f'Estimate {target_var_desc} of a given district in {country_name}.'
        
        file_path_var = file_path_mapping[target_var]

        gt = pd.read_csv(f'./data/labels/{target_ccode}_{file_path_var}{adm_offset}.csv', encoding='utf-8')[['area_id', col_var]]
        all_results_path = f'./data/all_features/{target_ccode}.csv'
        all_df = pd.read_csv(all_results_path)
        
        if target_var == 'GRDP' or target_var == 'population' or target_var == 'age_working_population':
            gt[col_var] = np.log(gt[col_var])
        elif target_var == 'age_old_dependency_ratio' or target_var == 'infant_mortality':
            gt[col_var] = (gt[col_var])/1000.0
        else:
            pass
        
        gpt35_results = []
        base_results = []
        no_results = []
        pos_results = []
        pos_neg_results = []
        pos_neg_mix_results = []
        
        for max_ensemble in max_ensemble_set:
    
            best_dict = {111: [], 222: [], 333: []}

            for random_seed in [111, 222, 333]:

                random.seed(random_seed)
                
                print(target_success_seed_list[target_index])
                print(target_success_ensemble_list[target_index])

                all_results = {}
                for success_id, (success_seed, success_ensemble_id) in enumerate (zip(target_success_seed_list[target_index], target_success_ensemble_list[target_index])):

                    print(f"Seed: {success_seed}, ensemble num: {success_ensemble_id}")
                    try_num = 0
                    while try_num < max_try_num:
#                         try:
                            module_dict = eval(answers_df[answers_df['seed'] == success_seed].iloc[success_ensemble_id]['module_dict'])

                            assert(len(module_dict) > 0)

                            module_columns = {}
                            for key, val in module_dict.items():
                                flag = False
                                s_columns = []
                                new_s_m = []
                                for s_m in val:
                                    if s_m == 'nightlight':
                                        s_columns.append('Nightlight_Sum')
                                        s_columns.append('Nightlight_Average')
                                        new_s_m.append('Nightlight_Sum')
                                        new_s_m.append('Nightlight_Average')
                                    elif s_m == 'neighbor_nightlight':
                                        s_columns.append('neighbor_Nightlight_Sum')
                                        s_columns.append('neighbor_Nightlight_Average')
                                        new_s_m.append('neighbor_Nightlight_Sum')
                                        new_s_m.append('neighbor_Nightlight_Average')
                                    else:
                                        s_columns.append(s_m)
                                        new_s_m.append(s_m)
                                    flag = True

                                module_dict[key] = new_s_m

                                if flag == False:
                                    print("error val")
                                    print(val)
                                    assert(0)
                                module_columns[key] = s_columns

                            address_df = all_df['address']
                            positive_df = all_df[module_columns['positive']]
                            negative_df = all_df[module_columns['negative']]
                            mixed_df = all_df[module_columns['mixed']]
                
                            positive_answers = eval(answers_df[answers_df['seed'] == success_seed].iloc[success_ensemble_id]['pos_list'])
                            negative_answers = eval(answers_df[answers_df['seed'] == success_seed].iloc[success_ensemble_id]['neg_list'])
                            mixed_answers = eval(answers_df[answers_df['seed'] == success_seed].iloc[success_ensemble_id]['mixed_list'])
    
                            if positive_answers == -1:
                                assert(0)
                            if negative_answers == -1:
                                assert(0)
                            if mixed_answers == -1:
                                assert(0)
                                
                            inter_positive_df = interaction(positive_answers, positive_df)
                            inter_negative_df = interaction(negative_answers, negative_df)
                            inter_mixed_df = interaction(mixed_answers, mixed_df)

                            inter_positive_df = inter_positive_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                            inter_negative_df = inter_negative_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                            inter_mixed_df = inter_mixed_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

                            df_all = pd.concat([positive_df, inter_positive_df, -negative_df, -inter_negative_df, mixed_df, inter_mixed_df], axis=1)
                            scalar = StandardScaler() 
                            df_all = scalar.fit_transform(df_all)

                            df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(df_all, gt[col_var].to_numpy(), train_size=num_shot, random_state=random_seed)
                            initial_coeffs = np.ones(df_all.shape[1]+1)
                            lambda_ridge = 1.0
                            bounds = []
                            for i in range(df_all.shape[1]+1):
                                if i < positive_df.shape[1] + inter_positive_df.shape[1] + negative_df.shape[1] + inter_negative_df.shape[1]:
                                    bounds.append((0, None))
                                else:
                                    bounds.append((None, None))
                                
                            temp_df_x_train = df_x_train
                            temp_df_y_train = df_y_train

                            val_results = []

                            if num_shot == 3:
                                arr = [0, 1, 2]
                            elif num_shot == 5:
                                arr = [0, 1, 2, 3, 4]
                            else:
                                assert(0)
                            
                            if num_valid == 2:
                                arr = itertools.permutations(arr, 2)
                                
                            for com in arr:
                
                                if num_valid == 1:
                                    sub_list = list([com]) # number of validation = 1
                                else: # if num_valid == 2:
                                    sub_list = list(com) # number of validation = 2
    
                                if num_shot == 3:
                                    num_shot = list(set([0, 1, 2]) - set(sub_list))
                                elif num_shot == 5:
                                    re_sub_list = list(set([0, 1, 2, 3, 4]) - set(sub_list))
                                else:
                                    assert(0)
                                
                                test_val_list = []
                                
                                # number of validation = 1
                                if num_valid == 1:
                                    df_x_val = temp_df_x_train[sub_list[0], :].reshape(1, -1) 
                                    df_y_val = temp_df_y_train[sub_list[0]].reshape(1, -1)

                                    if num_shot == 3:
                                        df_x_train = np.concatenate((temp_df_x_train[re_sub_list[0], :].reshape(1, -1), temp_df_x_train[re_sub_list[1], :].reshape(1, -1)), axis=0)
                                        df_y_train = np.concatenate((temp_df_y_train[re_sub_list[0]].reshape(1, -1), temp_df_y_train[re_sub_list[1]].reshape(1, -1)), axis=0)
                                    elif num_shot == 5:
                                        df_x_train = np.concatenate((temp_df_x_train[re_sub_list[0], :].reshape(1, -1), temp_df_x_train[re_sub_list[1], :].reshape(1, -1), temp_df_x_train[re_sub_list[2], :].reshape(1, -1), temp_df_x_train[re_sub_list[3], :].reshape(1, -1)), axis=0)
                                        df_y_train = np.concatenate((temp_df_y_train[re_sub_list[0]].reshape(1, -1), temp_df_y_train[re_sub_list[1]].reshape(1, -1), temp_df_y_train[re_sub_list[2]].reshape(1, -1), temp_df_y_train[re_sub_list[3]].reshape(1, -1)), axis=0)
                                    else:
                                        assert(0)
                                        
                                # number of validation = 2
                                if num_valid == 2:
                                    df_x_val = np.concatenate((temp_df_x_train[sub_list[0], :].reshape(1, -1), temp_df_x_train[sub_list[1], :].reshape(1, -1)), axis=0)
                                    df_y_val = np.concatenate((temp_df_y_train[sub_list[0]].reshape(1, -1), temp_df_y_train[sub_list[1]].reshape(1, -1)), axis=0)

                                    df_x_train = np.concatenate((temp_df_x_train[re_sub_list[0], :].reshape(1, -1), temp_df_x_train[re_sub_list[1], :].reshape(1, -1), temp_df_x_train[re_sub_list[2], :].reshape(1, -1)), axis=0)
                                    df_y_train = np.concatenate((temp_df_y_train[re_sub_list[0]].reshape(1, -1), temp_df_y_train[re_sub_list[1]].reshape(1, -1), temp_df_y_train[re_sub_list[2]].reshape(1, -1)), axis=0)
    
                                result = minimize(ridge_objective_function, initial_coeffs, args=(df_x_train, df_y_train, lambda_ridge), 
                                                  method='SLSQP', bounds=bounds, options={'maxiter': 2000})

                                if result.success == False:
                                    print("Optimization failed.")
                                    print("Reason:", result.message)
                                    
                                test_val = df_x_val @ result.x[:-1] + result.x[-1]
                                test_val = np.clip(test_val, 0, None)
                                test_val_list.append(test_val)

                                test_val_list = np.array(test_val_list)
                                test_val_final = test_val_list.mean(axis=0)

                                gtsup = pd.DataFrame(df_x_val)
                                gtsup['gt'] = df_y_val
                                gtsup['estimated'] = test_val_final

                                rmse = mean_squared_error(test_val_final, df_y_val, squared=False)
                                val_results.append(rmse)
            
                            all_results[success_id] = np.array(val_results).mean(axis=0)
                            break

                all_results = {k: v for k, v in sorted(all_results.items(), key=lambda item: item[1])}

                for i, (key, value) in enumerate(all_results.items()):
                    if i == 20:
                        break
                    best_dict[random_seed].append(key)
                    
                total_best_dict[target_var] = best_dict

[111, 111, 111, 111, 111, 111, 111, 222, 222, 222, 222, 222, 222, 222, 222, 222, 333, 333, 333, 333, 333, 333, 333, 333, 333, 333, 333, 333, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 555, 555, 555, 555, 555, 555, 555, 555, 555, 555, 555, 555, 555]
[3, 9, 11, 13, 14, 15, 17, 3, 7, 10, 12, 13, 14, 16, 17, 19, 0, 2, 3, 4, 5, 6, 8, 9, 12, 13, 18, 19, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 0, 2, 3, 7, 8, 10, 11, 12, 13, 14, 15, 16, 19]
Seed: 111, ensemble num: 3
Seed: 111, ensemble num: 9
Seed: 111, ensemble num: 11
Seed: 111, ensemble num: 13
Seed: 111, ensemble num: 14
Seed: 111, ensemble num: 15
Seed: 111, ensemble num: 17
Seed: 222, ensemble num: 3
Seed: 222, ensemble num: 7
Seed: 222, ensemble num: 10
Seed: 222, ensemble num: 12
Seed: 222, ensemble num: 13
Seed: 222, ensemble num: 14
Seed: 222, ensemble num: 16
Seed: 222, ensemble num: 17
Seed: 222, ensemble num: 19
Seed: 333, ensemble num: 0
Seed: 333, ensemble num: 2
Seed: 333, ensemble num: 3
Seed: 333, en

# Get results

In [24]:
total_best_dict

{'GRDP': {111: [49,
   27,
   26,
   36,
   18,
   22,
   43,
   14,
   10,
   28,
   2,
   3,
   41,
   20,
   31,
   47,
   29,
   51,
   24,
   11],
  222: [46,
   40,
   52,
   33,
   28,
   17,
   41,
   20,
   38,
   47,
   49,
   15,
   10,
   13,
   39,
   31,
   0,
   14,
   22,
   25],
  333: [19,
   48,
   12,
   26,
   28,
   27,
   16,
   33,
   30,
   45,
   53,
   21,
   31,
   17,
   11,
   7,
   14,
   46,
   20,
   0]}}

# Ours

In [30]:
selected_feature_importance_list = []
for ensemble_set in [1]:
    
    spearman_mean_list = []
    spearman_sem_list = []
    pearson_mean_list = []
    pearson_sem_list = []
    r2_mean_list = []
    r2_sem_list = []
    rmse_mean_list = []
    rmse_sem_list = []
        
    for target_index, target_var in enumerate (target_var_list):
    
        target_var = target_var_list[target_index]

        max_try_num = 10
        max_ensemble_set = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
        
        answers_path = f'./data/{target_dir}/{target_ccode}_{target_var}_extension_paragraph_output_25.csv'
        answers_df = pd.read_csv(answers_path)
        
        if target_ccode == 'VNM':
            adm_offset = '_adm1'
        else:
            adm_offset = ''

        if (target_ccode != 'KOR' and target_ccode != 'VNM') and target_var == 'GRDP':
            assert(0)
        elif target_ccode == 'KOR' and target_var == 'literacy_rate':
            assert(0)

        if target_var in list(mapping.keys()):
            col_var = mapping[target_var]
        else:
            col_var = target_var

        file_path_var = file_path_mapping[target_var]

        if target_var == 'literacy_rate' and target_ccode == 'VNM':
            col_var = 'LIT_15'
        elif target_var == 'literacy_rate' and target_ccode == 'KHM':
            col_var = 'LIT_T'

        target_var_desc = target_var_desc_mapping[target_var]
        country_name = country_name_mapping[target_ccode]
        
        Inference_desc = f'Estimate {target_var_desc} of a given district. Answer the numeric score only'
        TASK_DESC = f'Estimate {target_var_desc} of a given district in {country_name}.'
        
        file_path_var = file_path_mapping[target_var]

        gt = pd.read_csv(f'./data/labels/{target_ccode}_{file_path_var}{adm_offset}.csv', encoding='utf-8')[['area_id', col_var]]
        all_results_path = f'./data/all_features/{target_ccode}.csv'
        all_df = pd.read_csv(all_results_path)

        if target_var == 'GRDP' or target_var == 'population' or target_var == 'age_working_population':
            gt[col_var] = np.log(gt[col_var])
        elif target_var == 'age_old_dependency_ratio' or target_var == 'infant_mortality':
            gt[col_var] = (gt[col_var])/1000.0
        else:
            pass

        gpt35_results = []
        base_results = []
        no_results = []
        pos_results = []
        pos_neg_results = []
        pos_neg_mix_results = []
        
        total_ret_list = []
        total_ret_name_list = []
        
        for max_ensemble in max_ensemble_set:
    
            all_results = []
    
            selected_dict = total_best_dict[target_var]

            for random_seed in [111, 222, 333]:

                random.seed(random_seed)
                
                selected_id = selected_dict[random_seed]
                
                target_success_seed_value = []
                target_success_ensemble_value = []
                for ensemble_index in selected_id[:max_ensemble]:
                    target_success_seed_value.append(target_success_seed_list[target_index][ensemble_index])
                    target_success_ensemble_value.append(target_success_ensemble_list[target_index][ensemble_index])

                test_val_list = []
                for success_id, (success_seed, success_ensemble_id) in enumerate (zip(target_success_seed_value, target_success_ensemble_value)):
                    try_num = 0
                    feature_importance_list = []

                    while try_num < max_try_num:
#                         try:

                            module_dict = eval(answers_df[answers_df['seed'] == success_seed].iloc[success_ensemble_id]['module_dict'])

                            assert(len(module_dict) > 0)

                            module_columns = {}
                            for key, val in module_dict.items():
                                flag = False
                                s_columns = []
                                new_s_m = []
                                for s_m in val:
                                    if s_m == 'nightlight':
                                        s_columns.append('Nightlight_Sum')
                                        s_columns.append('Nightlight_Average')
                                        new_s_m.append('Nightlight_Sum')
                                        new_s_m.append('Nightlight_Average')
                                    elif s_m == 'neighbor_nightlight':
                                        s_columns.append('neighbor_Nightlight_Sum')
                                        s_columns.append('neighbor_Nightlight_Average')
                                        new_s_m.append('neighbor_Nightlight_Sum')
                                        new_s_m.append('neighbor_Nightlight_Average')
                                    else:
                                        s_columns.append(s_m)
                                        new_s_m.append(s_m)
                                    flag = True

                                module_dict[key] = new_s_m

                                if flag == False:
                                    print("error val")
                                    print(val)
                                    assert(0)
                                module_columns[key] = s_columns

                            address_df = all_df['address']
                            positive_df = all_df[module_columns['positive']]
                            negative_df = all_df[module_columns['negative']]
                            mixed_df = all_df[module_columns['mixed']]
                            
                            positive_len = positive_df.shape[1]
                            negative_len = negative_df.shape[1]
                            mixed_len = mixed_df.shape[1]
                            
                            positive_df = positive_df.loc[:,~positive_df.columns.duplicated()].copy()
                            negative_df = negative_df.loc[:,~negative_df.columns.duplicated()].copy()
                            mixed_df = mixed_df.loc[:,~mixed_df.columns.duplicated()].copy()
                                          
                            #########
                            
                            log_positive_df = np.log(positive_df)
                            sqrt_positive_df = np.sqrt(positive_df)
                            power_positive_df = np.power(positive_df, 2)
                            
                            log_positive_columns = []
                            sqrt_positive_columns = []
                            power_positive_columns = []
                            for init_column in positive_df.columns:
                                log_positive_columns.append(f'log({init_column})')
                                sqrt_positive_columns.append(f'sqrt({init_column})')
                                power_positive_columns.append(f'power({init_column})')
                            log_positive_df.columns = log_positive_columns
                            sqrt_positive_df.columns = sqrt_positive_columns
                            power_positive_df.columns = power_positive_columns
                        
                            log_positive_df = log_positive_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                            sqrt_positive_df = sqrt_positive_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                            power_positive_df = power_positive_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                        
                            log_negative_df = np.log(negative_df)
                            sqrt_negative_df = np.sqrt(negative_df)
                            power_negative_df = np.power(negative_df, 2)
                            
                            log_negative_columns = []
                            sqrt_negative_columns = []
                            power_negative_columns = []
                            for init_column in negative_df.columns:
                                log_negative_columns.append(f'log({init_column})')
                                sqrt_negative_columns.append(f'sqrt({init_column})')
                                power_negative_columns.append(f'power({init_column})')
                            log_negative_df.columns = log_negative_columns
                            sqrt_negative_df.columns = sqrt_negative_columns
                            power_negative_df.columns = power_negative_columns
                            
                            log_negative_df = log_negative_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                            sqrt_negative_df = sqrt_negative_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                            power_negative_df = power_negative_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                            
                            log_mixed_df = np.log(mixed_df)
                            sqrt_mixed_df = np.sqrt(mixed_df)
                            power_mixed_df = np.power(mixed_df, 2)
                            
                            log_mixed_columns = []
                            sqrt_mixed_columns = []
                            power_mixed_columns = []
                            for init_column in mixed_df.columns:
                                log_mixed_columns.append(f'log({init_column})')
                                sqrt_mixed_columns.append(f'sqrt({init_column})')
                                power_mixed_columns.append(f'power({init_column})')
                            log_mixed_df.columns = log_mixed_columns
                            sqrt_mixed_df.columns = sqrt_mixed_columns
                            power_mixed_df.columns = power_mixed_columns
                            
                            log_mixed_df = log_mixed_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                            sqrt_mixed_df = sqrt_mixed_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                            power_mixed_df = power_mixed_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

                            #########
                                   
                            all_modules = list(all_df.drop(['area_id', 'address', 'neighbor_address'], axis=1).columns)
                            ori_df = all_df[all_modules]
                            ori_df_len = ori_df.shape[1]
                            
                            positive_answers = eval(answers_df[answers_df['seed'] == success_seed].iloc[success_ensemble_id]['pos_list'])
                            negative_answers = eval(answers_df[answers_df['seed'] == success_seed].iloc[success_ensemble_id]['neg_list'])
                            mixed_answers = eval(answers_df[answers_df['seed'] == success_seed].iloc[success_ensemble_id]['mixed_list'])

                            inter_positive_df = interaction(positive_answers, positive_df)
                            inter_negative_df = interaction(negative_answers, negative_df)
                            inter_mixed_df = interaction(mixed_answers, mixed_df)

                            inter_positive_df = inter_positive_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                            inter_negative_df = inter_negative_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                            inter_mixed_df = inter_mixed_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
                            
                            inter_positive_df = inter_positive_df.T.drop_duplicates().T
                            inter_negative_df = inter_negative_df.T.drop_duplicates().T
                            inter_mixed_df = inter_mixed_df.T.drop_duplicates().T
                            
                            inter_positive_df_columns = inter_positive_df.columns
                            inter_negative_df_columns = inter_negative_df.columns
                            inter_mixed_df_columns = inter_mixed_df.columns
                            
                            pearson_filter = True
                            ##############
                            if pearson_filter == True:
                                temp_list = []
                                for feature in inter_positive_df_columns:
                                    temp = []
                                    for column in sorted(all_df.columns, reverse=True):
                                        if column in feature:
                                            feature = feature.replace(column, '')
                                            temp.append(column)
                                    temp_list.append(set(temp))

                                for (element, col) in zip(temp_list, inter_positive_df_columns):
                                    if len(element) == 1:
                                        inter_positive_df = inter_positive_df.drop([col], axis=1)

                                temp_list = []
                                for feature in inter_negative_df_columns:
                                    temp = []
                                    for column in sorted(all_df.columns, reverse=True):
                                        if column in feature:
                                            feature = feature.replace(column, '')
                                            temp.append(column)
                                    temp_list.append(set(temp))

                                for (element, col) in zip(temp_list, inter_negative_df_columns):
                                    if len(element) == 1:
                                        inter_negative_df = inter_negative_df.drop([col], axis=1)

                                temp_list = []
                                for feature in inter_mixed_df_columns:
                                    temp = []
                                    for column in sorted(all_df.columns, reverse=True):
                                        if column in feature:
                                            feature = feature.replace(column, '')
                                            temp.append(column)
                                    temp_list.append(set(temp))

                                for (element, col) in zip(temp_list, inter_mixed_df_columns):
                                    if len(element) == 1:
                                        inter_mixed_df = inter_mixed_df.drop([col], axis=1)
                                ##############        

                                inter_positive_df_len = inter_positive_df.shape[1]
                                inter_negative_df_len = inter_negative_df.shape[1]
                                inter_mixed_df_len = inter_mixed_df.shape[1]

                                inter_positive_df_columns = inter_positive_df.columns
                                inter_negative_df_columns = inter_negative_df.columns
                                inter_mixed_df_columns = inter_mixed_df.columns

                                df_total_all = pd.concat([ori_df, inter_positive_df, inter_negative_df, inter_mixed_df], axis=1)
                                scalar = StandardScaler() 
                                df_total_all = scalar.fit_transform(df_total_all)

                                ori_df = df_total_all[:,:ori_df_len]
                                inter_positive_df = df_total_all[:,ori_df_len:ori_df_len+inter_positive_df_len]
                                inter_negative_df = df_total_all[:,ori_df_len+inter_positive_df_len:ori_df_len+inter_positive_df_len+inter_negative_df_len]
                                inter_mixed_df = df_total_all[:,ori_df_len+inter_positive_df_len+inter_negative_df_len:]

                                ori_inter_positive_df = inter_positive_df
                                ori_inter_negative_df = inter_negative_df
                                ori_inter_mixed_df = inter_mixed_df

                                ori_inter_positive_df_columns = inter_positive_df_columns
                                ori_inter_negative_df_columns = inter_negative_df_columns
                                ori_inter_mixed_df_columns = inter_mixed_df_columns

                                # ****
                                for current in ['positive', 'negative', 'mixed']:
    #                             for current in ['negative']:

                                    top_k = 5
                                    option = 1
                                    option_percentage = 4
                                    if current == 'positive':
                                        inter_target_df = inter_positive_df
                                        inter_target_df_columns = inter_positive_df_columns
                                        if option == 0:
                                            if int(len(inter_positive_df_columns)) < top_k:
                                                target_len = int(len(inter_positive_df_columns))
                                            else:
                                                target_len = top_k
                                        else:
                                            target_len = int(len(inter_positive_df_columns)/option_percentage)#int(positive_len/4)
                                    elif current == 'negative':
                                        inter_target_df = inter_negative_df
                                        inter_target_df_columns = inter_negative_df_columns
                                        if option == 0:
                                            if int(len(inter_negative_df_columns)) < top_k:
                                                target_len = int(len(inter_negative_df_columns))
                                            else:
                                                target_len = top_k
                                        else:
                                            target_len = int(len(inter_negative_df_columns)/option_percentage)#int(negative_len/4)
                                    elif current == 'mixed':
                                        inter_target_df = inter_mixed_df
                                        inter_target_df_columns = inter_mixed_df_columns
                                        if option == 0:
                                            if int(len(inter_mixed_df_columns)) < top_k:
                                                target_len = int(len(inter_mixed_df_columns))
                                            else:
                                                target_len = top_k
                                        else:
                                            target_len = int(len(inter_mixed_df_columns)/option_percentage)#int(mixed_len/4)

                                    pearson_dict = {}
                                    p_ori_df = ori_df
                                    p_inter_target_df = inter_target_df
                                    flag_array = np.full((inter_target_df.shape[1]), False)
    #                                 print(flag_array.shape)
    #                                 assert(0)
                                    for p_index in range(inter_target_df.shape[1]):
                                        mini_pearson_dict = {}

                                        for _target_index in range(inter_target_df.shape[1]):
                                            target_pearson = []

                                            for ori_index in range(p_ori_df.shape[1]):
                                                target_pearson.append(stats.pearsonr(p_inter_target_df[:,_target_index].reshape(-1, 1).T.tolist()[0], p_ori_df[:,ori_index].reshape(-1, 1).T.tolist()[0]).statistic)

                                            if flag_array[_target_index] == True:
                                                mini_pearson_dict[(_target_index, inter_target_df_columns[_target_index])] = 1.0
                                            else:

                                                target_pearson = np.array(target_pearson)
                                                if current == 'positive' or current == 'negative':
                                                    target_pearson = target_pearson[target_pearson > 0.0]
                                                    target_pearson = target_pearson.mean()
                                                else:
                                                    target_pearson = target_pearson.mean()

                                                mini_pearson_dict[(_target_index, inter_target_df_columns[_target_index])] = target_pearson

                                        mini_pearson_dict = sorted(mini_pearson_dict.items(), key=lambda x:x[1]) # 낮은 순

                                        ((_target_index, _), _) = mini_pearson_dict[:1][0]

                                        flag_array[_target_index] = True

                                        new_inter_target_df = np.zeros(inter_target_df[:,:1].shape)
                                        new_inter_target_df_columns = []

                                        for pi, ((_target_index, target_name), val) in enumerate(mini_pearson_dict[:1]):
                                            new_inter_target_df[:,pi] = inter_target_df[:,_target_index]
                                            new_inter_target_df_columns.append(target_name)

                                        if current == 'positive':
                                            inter_positive_df = new_inter_target_df
                                            inter_positive_df_columns = new_inter_target_df_columns
                                            addition_target_df = inter_positive_df
                                        elif current == 'negative':
                                            inter_negative_df = new_inter_target_df
                                            inter_negative_df_columns = new_inter_target_df_columns
                                            addition_target_df = inter_negative_df
                                        elif current == 'mixed':
                                            inter_mixed_df = new_inter_target_df
                                            inter_mixed_df_columns = new_inter_target_df_columns
                                            addition_target_df = inter_mixed_df

                                        p_ori_df = np.concatenate([p_ori_df, addition_target_df], axis=1)
                                        ((top1_index, top1_name), top1_val) = mini_pearson_dict[:1][0]
                                        pearson_dict[(p_index, top1_index, top1_name)] = top1_val

                                    pearson_dict = sorted(pearson_dict.items(), key=lambda x:x[1], reverse=True)

                                    ############################
                                    if current == 'positive':
                                        inter_target_df = ori_inter_positive_df
                                        inter_target_df_columns = ori_inter_positive_df_columns
                                    elif current == 'negative':
                                        inter_target_df = ori_inter_negative_df
                                        inter_target_df_columns = ori_inter_negative_df_columns
                                    elif current == 'mixed':
                                        inter_target_df = ori_inter_mixed_df
                                        inter_target_df_columns = ori_inter_mixed_df_columns
                                    ############################

                                    new_inter_target_df = np.zeros(inter_target_df[:,:target_len].shape)
                                    new_inter_target_df_columns = []
                                    for pi, ((p_index, _target_index, target_name), val) in enumerate(pearson_dict[:target_len]):
                                        new_inter_target_df[:,pi] = inter_target_df[:,_target_index]
                                        new_inter_target_df_columns.append(target_name)

                                    if current == 'positive':
                                        inter_positive_df = new_inter_target_df
                                        inter_positive_df_columns = new_inter_target_df_columns
                                    elif current == 'negative':
                                        inter_negative_df = new_inter_target_df
                                        inter_negative_df_columns = new_inter_target_df_columns
                                    elif current == 'mixed':
                                        inter_mixed_df = new_inter_target_df
                                        inter_mixed_df_columns = new_inter_target_df_columns
                                # ****
                    
                            temp_all_df = pd.DataFrame([])
                            temp_length = 0
                            if inter_positive_df.shape[1] != 0:
                                inter_positive_df = pd.DataFrame(inter_positive_df)
                                inter_positive_df.columns = list(inter_positive_df_columns)
                                inter_positive_df = inter_positive_df.loc[:,~inter_positive_df.columns.duplicated()].copy()
                                temp_all_df = pd.concat([temp_all_df, positive_df, log_positive_df, sqrt_positive_df, power_positive_df, inter_positive_df], axis=1)
                                temp_length = temp_length + positive_df.shape[1]*4 + inter_positive_df.shape[1]
                            else:
                                temp_all_df = pd.concat([temp_all_df, positive_df, log_positive_df, sqrt_positive_df, power_positive_df], axis=1)
                                temp_length = temp_length + positive_df.shape[1]*4
                            
                            if inter_negative_df.shape[1] != 0:
                                inter_negative_df = pd.DataFrame(inter_negative_df)
                                inter_negative_df.columns = list(inter_negative_df_columns)
                                inter_negative_df = inter_negative_df.loc[:,~inter_negative_df.columns.duplicated()].copy()
                                temp_all_df = pd.concat([temp_all_df, -negative_df, -log_negative_df, -sqrt_negative_df, -power_negative_df, -inter_negative_df], axis=1)
                                temp_length = temp_length + negative_df.shape[1]*4 + inter_negative_df.shape[1]
                            else:
                                temp_all_df = pd.concat([temp_all_df, -negative_df, -log_negative_df, -sqrt_negative_df, -power_negative_df], axis=1)
                                temp_length = temp_length + negative_df.shape[1]*4
                            
                            if inter_mixed_df.shape[1] != 0:
                                inter_mixed_df = pd.DataFrame(inter_mixed_df)
                                inter_mixed_df.columns = list(inter_mixed_df_columns)
                                inter_mixed_df = inter_mixed_df.loc[:,~inter_mixed_df.columns.duplicated()].copy()
                                temp_all_df = pd.concat([temp_all_df, mixed_df, log_mixed_df, sqrt_mixed_df, power_mixed_df, inter_mixed_df], axis=1)

                            else:
                                temp_all_df = pd.concat([temp_all_df, mixed_df, log_mixed_df, sqrt_mixed_df, power_mixed_df], axis=1)

                            df_all = temp_all_df
                            
                            scalar = StandardScaler() 
                            df_all = scalar.fit_transform(df_all)

                            df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(df_all, gt[col_var].to_numpy(), train_size=num_shot, random_state=random_seed)
                            initial_coeffs = np.ones(df_all.shape[1]+1)
                            lambda_ridge = 1.0
                            bounds = []
                            for i in range(df_all.shape[1]+1):
                                if i < temp_length:
                                    bounds.append((0, None))
                                else:
                                    bounds.append((None, None))

                            result = minimize(ridge_objective_function, initial_coeffs, args=(df_x_train, df_y_train, lambda_ridge), 
                                              method='SLSQP', bounds=bounds, options={'maxiter': 2000})

                            if result.success == False:
                                print("Optimization failed.")
                                print("Reason:", result.message)

                            break

                    test_val = df_x_test @ result.x[:-1] + result.x[-1]
                    test_val = np.clip(test_val, 0, None)
                    test_val_list.append(test_val)

                test_val_list = np.array(test_val_list)
                test_val_final = test_val_list.mean(axis=0)

                gtsup = pd.DataFrame(df_x_test)
                gtsup['gt'] = df_y_test
                gtsup['estimated'] = test_val_final

                spearman = gtsup[['gt', 'estimated']].corr(method='spearman').iloc[0][1]
                pearson = gtsup[['gt', 'estimated']].corr(method='pearson').iloc[0][1]    
                r2 = r2_score(df_y_test, test_val_final)
                rmse = mean_squared_error(test_val_final, df_y_test, squared=False) 
                all_results.append([spearman, pearson, r2, rmse])

            all_results_df = pd.DataFrame(all_results)
            all_results_df.columns = ['spearman', 'pearson', 'r2', 'RMSE']

            pos_neg_mix_results.append((all_results_df.mean(), all_results_df.sem()))

        for i in range (len(max_ensemble_set)):

            print(f'ensemble: {max_ensemble_set[i]}')
            print(pos_neg_mix_results[i][0][0], pos_neg_mix_results[i][0][1], pos_neg_mix_results[i][0][2], pos_neg_mix_results[i][0][3])
            print(pos_neg_mix_results[i][1][0], pos_neg_mix_results[i][1][1], pos_neg_mix_results[i][1][2], pos_neg_mix_results[i][1][3])

            spearman_mean_list.append(pos_neg_mix_results[i][0][0])
            spearman_sem_list.append(pos_neg_mix_results[i][1][0])
            pearson_mean_list.append(pos_neg_mix_results[i][0][1])
            pearson_sem_list.append(pos_neg_mix_results[i][1][1])
            r2_mean_list.append(pos_neg_mix_results[i][0][2])
            r2_sem_list.append(pos_neg_mix_results[i][1][2])
            rmse_mean_list.append(pos_neg_mix_results[i][0][3])
            rmse_sem_list.append(pos_neg_mix_results[i][1][3])

            print('')

/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/pyth

/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/pyth

/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/kyeongjin/.conda/envs/python38/lib/pyth